In [5]:
include("../SCLUtility.jl")
include("../AWGN.jl")
include("../Polar.jl")
include("../SCL.jl")
struct SCLProblem
    N::Int
    K::Int
    pm::PathManager
    p::Polar
end

SCLProblem(N,K,L) = begin
    p = Polar(BattGN,N,K)
    pm = PathManager(N,L)
    SCLProblem(N,K,pm,p)
end

@inline function initialize!(t::SCLProblem,a::AWGN)
    updatellr!(a)
    initialize!(t.pm,a.llr)
    nothing
end

function solver(sim,t::SCLProblem,a::AWGN)
    v = [in(i,t.p.F) for i=1:t.N]
    @inline isfrozen(i)  = v[i]
    bler = 0; ber = 0
    for i=1:sim
        initialize!(t,a)
        p = SCL(t.pm,isfrozen)
        tmp = errors(t.pm,p)
        if !iszero(tmp)
            bler += 1
            ber += tmp
        end
    end
    return bler/sim,ber/sim/t.K
end

solver (generic function with 1 method)

In [7]:
_N = 1024; _K = 512; _L = 1
t = SCLProblem(_N,_K,_L)
a = AWGN(_N,_K,8,3.0)
@time solver(300,t,a)

  0.224757 seconds (260.67 k allocations: 5.446 MiB, 51.83% compilation time)


(0.0033333333333333335, 0.0003385416666666667)

In [4]:
N = 1024; K = 512; L = 32
t = SCLProblem(N,K,L)
a = AWGN(N,K,8,3.0)
@time solver(30000,t,a)

799.463376 seconds (92.47 M allocations: 10.761 GiB, 0.06% gc time, 0.05% compilation time)


(0.0003333333333333333, 1.4583333333333333e-5)

In [2]:
N = 2048; K = 1024; L = 4
t = SCLProblem(N,K,L)
a = AWGN(N,K,8,2.0)
@time solver(1000,t,a)

  5.588407 seconds (6.98 M allocations: 203.169 MiB, 0.43% gc time, 7.25% compilation time)


(0.002, 0.00021875)

In [34]:
N = 1024; K = 512; L = 4
t = SCLProblem(N,K,L)
a = AWGN(N,K,8,2.0)
@time solver(1000,t,a)

  1.914409 seconds (2.57 M allocations: 86.274 MiB, 0.30% gc time)


(0.013, 0.001716796875)

In [13]:
t.pm.patharr[2].C
#t.pm.patharr[2].path

3-element Vector{Vector{Vector{Int64}}}:
 [[1, 0], [1, 0], [0, 0], [0, 0]]
 [[0, 1], [0, 0]]
 [[1, 0]]

In [179]:
N = 1024; K = 512; L = 4
t = SCLProblem(N,K,L)
a = AWGN(N,K,8,2.0)
@time solver(1,t,a)

phai 1
phai 1
phai 1
phai 1
  0.456459 seconds (59.58 k allocations: 3.673 MiB, 5.67% gc time, 99.62% compilation time)


(0.0, 0.0)

In [173]:
println("すべてゼロになるのはおかしい")
x->isequal(1)(x[1]),t.pm.patharr[3].C[1] |> println
findall(x->isequal(1)(x[1]),t.pm.patharr[3].C[1]) |> println
findall(isequal(1),t.pm.patharr[3].path) |> println

すべてゼロになるのはおかしい
Vector{Bool}[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 

In [125]:
N = 1024; K = 512; L = 4
t = SCLProblem(N,K,L)
a = AWGN(N,K,8,2.0)
@time solver(40,t,a) 

  0.072038 seconds (102.76 k allocations: 3.452 MiB)


(0.1, 0.002197265625)

In [121]:
t.pm.patharr[1].C[1] |> println
t.pm.patharr[4].path |> println

Vector{Bool}[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0,

In [31]:
N = 1024; K = 512; L = 1
t = SCLProblem(N,K,L)
a = AWGN(N,K,8,2.0)
@time solver(1,t,a)

  0.000598 seconds (2.57 k allocations: 89.406 KiB)


(0.0, 0.0)

In [207]:
N = 2048; K = 1024; L = 2
t = SCLProblem(N,K,L)
a = AWGN(N,K,8,3.0)
@time solver(1,t,a)

(0.0, 0.0)

In [208]:
println(getpathpath(t.pm,1))

In [209]:
N = 256; K = 128
t = SCLProblem(N,K,1)
a = AWGN(N,K,8,2.0)
solver(30000,t,a)

(0.12913333333333332, 0.012919270833333333)

In [210]:
N = 256; K = 128
t = SCLProblem(N,K,1)
a = AWGN(N,K,8,3.0)
solver(100000,t,a)

(0.01267, 0.000896484375)

In [211]:
#L=1なのでSC復号
N = 1024; K = 512
t = SCLProblem(N,K,1)
a = AWGN(N,K,8,3.5)
solver(90000,t,a)

(0.00014444444444444444, 4.253472222222222e-6)

In [212]:
N = 256; K = 128
t = SCLProblem(N,K,1)
a = AWGN(N,K,8,4.0)
solver(100000,t,a)

(0.00059, 3.140625e-5)

In [213]:
N = 2048; K = 1024; L = 1
t = SCLProblem(N,K,L)
a = AWGN(N,K,8,3.0)
solver(100000,t,a)

(0.00026, 4.90234375e-6)

# 

In [214]:
N = 2048; K = 1024; L = 2
t = SCLProblem(N,K,L)
a = AWGN(N,K,8,2.0)
solver(1000,t,a)

(0.067, 0.0117001953125)

In [215]:
N = 2048; K = 1024; L = 2
t = SCLProblem(N,K,L)
a = AWGN(N,K,8,2.0)
solver(1,t,a)

(0.0, 0.0)

In [19]:
open("exe.txt","w") do f
    println(f,"""
    今日の天気は晴れ。
    これから大学で送別会がある
    """)
end

In [20]:
using Dates
today() |> println
now() |> println

2024-09-30
2024-09-30T13:27:34.394


In [25]:
Dict([("A",1),("B",2)])

Dict{String, Int64} with 2 entries:
  "B" => 2
  "A" => 1

# key-value型を使いこなす

In [22]:
#keyとvalueを持つDictionary型
#keyというオブジェクトとvalueというオブジェクト、dictはその対応である
d = Dict("N"=>1024,"K"=>512,"sim"=>1000,"eb"=>3.0)
#Vectorと同じ
d["N"]
keys(d) |> println
values(d) |> println

["sim", "N", "eb", "K"]
Real[1000, 1024, 3.0, 512]


In [30]:
map(i->2i,[1:9;]) |> println

[2, 4, 6, 8, 10, 12, 14, 16, 18]


# deconstruct 集まったものを分解する

In [33]:
a,b,c = 1:3
_,d = 1:10

2

In [ ]:
args = [1000,100,3.0]
d = Dict(:N=>10,:K=>512,:eb=3.0)

# parseargument
- 引数は構造的に記述されているので、解析することができる

In [35]:
using ArgParse

function parse_commandline()
    s = ArgParseSettings()

    @add_arg_table! s begin
        "--name", "-n"
            help = "名前"
            required = true
        "--age", "-a"
            help = "年齢"
            arg_type = Int
            default = 30
        "--city", "-c"
            help = "都市"
            default = "Tokyo"
        "--verbose", "-v"
            help = "詳細モードを有効にする"
            action = :store_true
    end

    return parse_args(s)
end

function main()
    parsed_args = parse_commandline()
    println("Parsed args:")
    for (arg,val) in parsed_args
        println("  $arg  =>  $val")
    end

    # 引数の使用例
    name = parsed_args["name"]
    age = parsed_args["age"]
    city = parsed_args["city"]
    verbose = parsed_args["verbose"]

    println("\n処理結果:")
    println("こんにちは、$(name)さん！")
    println("あなたは$(age)歳で、$(city)に住んでいますね。")
    if verbose
        println("詳細モードが有効です。より多くの情報が表示されます。")
    end
end

main()